In [ ]:
import sys, os
sys.path.append(os.path.dirname(os.path.dirname(os.path.dirname(os.path.realpath('__file__')))))
from src.pricer import *

# European Tree

In [ ]:
# Create European contract

MarketData.initialize()
und = Stock.TEST_COMPANY
expiry = 1.0
strike = 1.0 * MarketData.get_spot()[und]
europeanContract = EuropeanContract(und, PutCallFwd.CALL, LongShort.LONG, strike, expiry)

In [ ]:
# Create European Tree pricer

params = TreeParams(2)
model = FlatVolModel(und)
europeanPricer = EuropeanTreePricer(europeanContract, model, params)

In [ ]:
europeanPricer.calc_fair_value()

In [ ]:
# European Analytic pricer

analyticPricer = EuropeanAnalyticPricer(europeanContract, model, Params())
analyticPricer.calc_fair_value()

### Tree price != Analytic price, is it a bug?

In [ ]:
params100 = TreeParams(100)
europeanPricer100 = EuropeanTreePricer(europeanContract, model, params100)
europeanPricer100.calc_fair_value()

In [ ]:
# Calibrating European Tree

paramsCalib = TreeParams(2,0.2)
europeanPricerCalib = EuropeanTreePricer(europeanContract, model, paramsCalib)
europeanPricerCalib.calc_fair_value()

# American Tree

In [ ]:
# American contract and Tree pricer

americanContract = AmericanContract(und, PutCallFwd.CALL, LongShort.LONG, strike, expiry)
amPricer = AmericanTreePricer(americanContract, model, params)
amPricer.calc_fair_value()

### American call price = European call price, is it a bug?

# Inbalanced Tree

In [ ]:
# European Tree with step size specified

inbalancedParams = TreeParams(2, np.nan, 1.2, 0.8)
inbalancedEuropeanPricer = EuropeanTreePricer(europeanContract, model, inbalancedParams)
inbalancedEuropeanPricer.calc_fair_value()

In [ ]:
# American contract and Tree pricer

inbalancedAmPricer = AmericanTreePricer(americanContract, model, inbalancedParams)
inbalancedAmPricer.calc_fair_value()

### American call price != European call price, is it a bug?

In [ ]:
#import numpy as np

def binomial_tree_pricer(option_type, S0, K, T, r, u, d, n):
    """
    Prices an option using the binomial tree model.
    
    Parameters:
    - option_type: 'call' or 'put'
    - S0: Initial stock price
    - K: Strike price
    - T: Time to maturity (in years)
    - r: Risk-free interest rate
    - u: Factor by which the price goes up
    - d: Factor by which the price goes down
    - n: Number of steps in the tree
    """
    dt = T / n  # time step
    q = (np.exp(r * dt) - d) / (u - d)  # risk-neutral probability

    # Price tree
    price_tree = np.zeros((n + 1, n + 1))
    for i in range(n + 1):
        for j in range(i + 1):
            price_tree[j, i] = S0 * (u ** j) * (d ** (i - j))

    # Option value at maturity
    option_tree = np.zeros((n + 1, n + 1))
    for j in range(n + 1):
        if option_type == "call":
            option_tree[j, n] = max(0, price_tree[j, n] - K)
        elif option_type == "put":
            option_tree[j, n] = max(0, K - price_tree[j, n])

    # Calculate option price at t=0
    for i in range(n - 1, -1, -1):
        for j in range(i + 1):
            option_tree[j, i] = np.exp(-r * dt) * (q * option_tree[j, i + 1] + (1 - q) * option_tree[j + 1, i + 1])

    return option_tree[0, 0]


# HOMEWORK IMPLEMENTATION: Task 
# Calibrater for Tree

In [ ]:
def calibrate_step_size(target_price, option_type, S0, K, T, r, n, tol=1e-3):
    """
    Calibrates the step size for the binomial tree to match a target option price.

    Parameters:
    - target_price: The market price of the option
    - option_type: 'call' or 'put'
    - S0: Initial stock price
    - K: Strike price
    - T: Time to maturity (in years)
    - r: Risk-free interest rate
    - n: Number of steps in the tree
    - tol: Tolerance for the price difference
    """
    u = 1.0  # initial guess for up factor
    increment = 0.05  # initial step size for searching

    while True:
        d = 1 / u  # down factor is inverse of up factor
        price = binomial_tree_pricer(option_type, S0, K, T, r, u, d, n)
        
        if abs(price - target_price) < tol:
            return u, d
        elif price < target_price:
            u += increment
        else:
            u -= increment
            increment /= 2  # reducing increment to refine the search


In [ ]:
#import numpy as np
from scipy.optimize import minimize
from scipy.stats import norm


In [ ]:
def black_scholes_call(S, K, T, r, vol):
    """Calculate European call option price using Black-Scholes formula."""
    d1 = (np.log(S / K) + (r + 0.5 * vol ** 2) * T) / (vol * np.sqrt(T))
    d2 = d1 - vol * np.sqrt(T)
    call_price = S * norm.cdf(d1) - K * np.exp(-r * T) * norm.cdf(d2)
    return call_price


In [ ]:
def binomial_tree_call(S, K, T, r, vol, nr_steps):
    """
    Calculate European call option price using a Balanced Binomial Tree.
    
    Parameters:
    - S: Current stock price
    - K: Strike price
    - T: Time to maturity
    - r: Risk-free interest rate
    - vol: Volatility of the underlying asset
    - nr_steps: Number of steps in the binomial tree
    """
    dt = T / nr_steps  # Time step
    u = np.exp(vol * np.sqrt(dt))  # Up factor
    d = 1 / u  # Down factor
    q = (np.exp(r * dt) - d) / (u - d)  # Risk-neutral probability

    # Step 2: Construct the binomial tree
    price_tree = np.zeros((nr_steps + 1, nr_steps + 1))
    for i in range(nr_steps + 1):
        for j in range(i + 1):
            price_tree[j, i] = S * (u ** j) * (d ** (i - j))

    # Step 3: Calculate option value at maturity
    option_tree = np.zeros((nr_steps + 1, nr_steps + 1))
    for j in range(nr_steps + 1):
        option_tree[j, nr_steps] = max(0, price_tree[j, nr_steps] - K)

    # Step 4: Backward induction to determine the option price at t=0
    for i in range(nr_steps - 1, -1, -1):
        for j in range(i + 1):
            option_tree[j, i] = np.exp(-r * dt) * (q * option_tree[j, i + 1] + (1 - q) * option_tree[j + 1, i + 1])

    return option_tree[0, 0]


In [ ]:
def calibrate_vol(S, K, T, r, target_price, nr_steps):
    """
    Calibrate the volatility for the BalancedBinomialTree model.
    
    Parameters:
    - S: Current stock price
    - K: Strike price
    - T: Time to maturity
    - r: Risk-free interest rate
    - target_price: Target option price (from Black-Scholes)
    - nr_steps: Number of steps in the binomial tree
    """
    def objective(vol):
        # Calculate the price difference between Black-Scholes and Binomial Tree
        tree_price = binomial_tree_call(S, K, T, r, vol, nr_steps)
        return (tree_price - target_price) ** 2

    # Initial guess for volatility
    vol_init = 0.2

    result = minimize(objective, vol_init, method='Nelder-Mead')

    if result.success:
        return result.x[0]  # calibrated volatility
    else:
        raise ValueError("Calibration failed")


HOMEWORK IMPLEMENTATION : TASK TreePricer

In [ ]:
def up_and_out_call_binomial_tree(S0, K, T, r, vol, barrier, nr_steps):
    """
    Prices an up-and-out call option using a binomial tree.
    
    Parameters:
    - S0: Initial stock price
    - K: Strike price
    - T: Time to maturity
    - r: Risk-free interest rate
    - vol: Volatility of the underlying asset
    - barrier: Barrier level
    - nr_steps: Number of steps in the tree
    """
    dt = T / nr_steps  # Time step
    u = np.exp(vol * np.sqrt(dt))  # Up factor
    d = 1 / u  # Down factor
    q = (np.exp(r * dt) - d) / (u - d)  # Risk-neutral probability

    # Construct the price tree
    price_tree = np.zeros((nr_steps + 1, nr_steps + 1))
    for i in range(nr_steps + 1):
        for j in range(i + 1):
            price_tree[j, i] = S0 * (u ** j) * (d ** (i - j))
            if price_tree[j, i] >= barrier:
                price_tree[j, i] = 0  # Option becomes worthless if barrier is hit

    # Option values at maturity
    option_tree = np.zeros((nr_steps + 1, nr_steps + 1))
    for j in range(nr_steps + 1):
        option_tree[j, nr_steps] = max(0, price_tree[j, nr_steps] - K)

    # Backward induction
    for i in range(nr_steps - 1, -1, -1):
        for j in range(i + 1):
            option_tree[j, i] = np.exp(-r * dt) * (q * option_tree[j, i + 1] + (1 - q) * option_tree[j + 1, i + 1])
            if price_tree[j, i] == 0:
                option_tree[j, i] = 0  # Ensure option value is zero if barrier is hit

    return option_tree[0, 0]


In [ ]:
class UpAndOutCallTreePricer(TreePricer):
    def __init__(self, tree_params, barrier):
        super().__init__(tree_params)
        self.barrier = barrier

    def pre_final_value(self, S, K, r, dt, u, d, q):
        # Creating an array to store the option values at the next time step
        next_step_values = np.zeros(len(S))

        # Iterate over each node in the current step
        for i in range(len(S)):
            up_price = S[i] * u
            down_price = S[i] * d

            # Check if the price breaches the barrier in either direction
            if up_price >= self.barrier or down_price >= self.barrier:
                # If the barrier is breached, the option value becomes zero
                next_step_values[i] = 0
            else:
                # Calculate the option value using risk-neutral probabilities
                up_value = max(0, up_price - K)
                down_value = max(0, down_price - K)
                next_step_values[i] = np.exp(-r * dt) * (q * up_value + (1 - q) * down_value)

        return next_step_values


In [ ]:
# Parameters
S0 = 100  # Spot price
K = S0    # Strike = spot
T = 1     # Time to maturity = 1 year
r = 0.05  # Risk-free interest rate
vol = 0.2 # Volatility
nr_steps = 252
barrier_levels = [1.1 * S0, 1.2 * S0, 1.3 * S0]  # Different barrier levels

# Comparing prices
for barrier in barrier_levels:
    tree_price = up_and_out_call_binomial_tree(S0, K, T, r, vol, barrier, nr_steps)
    mc_price = # Monte Carlo simulation here
    print(f"Barrier: {barrier}, Tree Price: {tree_price}, MC Price: {mc_price}")
